In [3]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('../dataset/x_train.npy')
    x_train2 = x_train + np.random.rand(60000,28,28)
    x_train = np.concatenate([x_train,x_train2])
    y_train = np.load('../dataset/y_train.npy')
    y_train = np.concatenate([y_train,y_train])
    
    # テストデータ
    x_test = np.load('../dataset/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

x_train, y_train, x_test = load_fashionmnist()

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=1000)

def np_log(x):
    return np.log(np.clip(x, 1e-10, x))

def relu(x):
    return np.maximum(x, 0)

def deriv_relu(x):
    return (x > 0).astype(x.dtype)

def softmax(x):
    x -= x.max(axis=1, keepdims=True)
    x_exp = np.exp(x)
    return x_exp / np.sum(x_exp, axis=1, keepdims=True)

def deriv_softmax(x):
    return softmax() * (1 - softmax(x))

def f_props(layers, x):
    for layer in layers:
        x = layer(x)
    return x
def b_props(layers, delta):
    batch_size = delta.shape[0]
    
    for i, layer in enumerate(layers[::-1]):
        if i == 0: # 出力層の場合
            layer.delta = delta # y - t
            layer.compute_grad() # 勾配の計算
        else: # 出力層以外の場合
            delta = layer.b_prop(delta, W) # 逆伝播
            layer.compute_grad() # 勾配の計算

        W = layer.W
        
layers = [
    Dense(784, 100, relu, deriv_relu),
    Dense(100, 100, relu, deriv_relu),
    Dense(100, 10, softmax, deriv_softmax)
]

def train_mnist(x, t, eps=0.01):
    # WRITE ME 注意、epsは手書き！
    # 順伝播
    y = f_props(layers, x)

    # 誤差の計算
    cost = (- t * np_log(y)).sum(axis=1).mean()
    
    # 逆伝播
    delta = y - t
    b_props(layers, delta)

    # パラメータの更新
    update_params(layers, eps)

    return cost
def valid_mst(x):
    # 順伝播
    y = f_props(layers, x)
    
    # 誤差の計算
    #cost = (- t * np_log(y)).sum(axis=1).mean()
    
    return y
    
def update_params(layers, eps):
    for layer in layers:
        layer.W -= eps * layer.dW
        layer.b -= eps * layer.db
        
for epoch in range(5):
    x_train, y_train = shuffle(x_train, y_train)
    i=0
    for x, t in zip(x_train, y_train):
        cost = train_mnist(x[None, :], t[None, :], eps=0.01)
    
    #cost, y_pred = valid_mst(x_valid, y_valid)
    y_pred = valid_mst(x_test)
    print(i)
    #accuracy = accuracy_score(y_valid.argmax(axis=1), y_pred.argmax(axis=1))
    #print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(epoch + 1, cost, accuracy))
    
y_pred = np.argmax(y_pred, axis=1)
print(len(y_pred))
submission = pd.Series(y_pred, name='label')
submission.to_csv('../dataset/submission_pred.csv', header=True, index_label='id')

0
0
0
0
0
10000
